# Symbolic computations with ABS multiforms

Code related to:
JJ Richardson, M vermeeren. "Discrete Lagrangian multiforms for ABSequations I: quad equations"

### Implementing class for Quad equations

In [1]:
import numpy

var("u,u_i,u_j,u_k,u_ij,u_jk,u_ik,u_ijk")
var("a_i,a_j,a_k")
var("delta")
varlist = [u,u_i,u_j,u_k,u_ij,u_jk,u_ik,u_ijk]
paramvarlist = [a_i,a_j,a_k,u,u_i,u_j,u_k,u_ij,u_jk,u_ik,u_ijk]

def cycle(func):
    """symmerise wrt cyclic permuations"""
    var("temp")
    func = func.subs(a_k == temp).subs(a_j == a_k).subs(a_i == a_j).subs(temp == a_i)
    func = func.subs(u_k == temp).subs(u_j == u_k).subs(u_i == u_j).subs(temp == u_i)
    func = func.subs(u_ij == temp).subs(u_ik == u_ij).subs(u_jk == u_ik).subs(temp == u_jk)
    return func

def reverse(func):
    """symmerise wrt cyclic permuations"""
    var("temp")
    func = func.subs(u == temp).subs(u_ijk == u).subs(temp = u_ijk)
    func = func.subs(u_i == temp).subs(u_jk == u_i).subs(temp = u_jk)
    func = func.subs(u_j == temp).subs(u_ik == u_j).subs(temp = u_ik)
    func = func.subs(u_k == temp).subs(u_ij == u_k).subs(temp = u_ij)
    return func

def cyclic(func):
    """symmerise wrt cyclic permuations"""
    var("temp")
    out = func
    out += cycle(func)
    out += cycle(cycle(func))
    return out

def shiftk(func):
    func = func.subs(u == u_k)
    func = func.subs(u_i == u_ik)
    func = func.subs(u_j == u_jk)
    func = func.subs(u_ij == u_ijk)
    return func

def allquads(quad):
    return [0,reverse(cycle(quad)),reverse(cycle(cycle(quad))),reverse(quad),quad,cycle(quad),cycle(cycle(quad)),0]

def crandom(real = False):
    return numpy.random.uniform(-1, 1) + (1-int(real))*I * numpy.random.uniform(-1, 1)

def randomtest(func, real = False):
    """test whether something is zero by assigning random values to variables"""
    out = []
    for i in range(5):
        test = func.subs(a_k == crandom(real)).subs(a_j == crandom(real)).subs(a_i == crandom(real))
        test = test.subs(u == crandom(real)).subs(u_k == crandom(real)).subs(u_j == crandom(real)).subs(u_i == crandom(real))
        test = test.subs(u_ij == crandom(real)).subs(u_ik == crandom(real)).subs(u_jk == crandom(real)).subs(u_ijk == crandom(real))
        out += [test]
    return out

def randominit(real = False, lim=5):
    sol = {u: numpy.random.uniform(-lim, lim) + (1-int(real))*1.j * numpy.random.uniform(-lim, lim)}
    sol[u_i] = numpy.random.uniform(-lim, lim) + (1-int(real))*1.j * numpy.random.uniform(-lim, lim)
    sol[u_j] = numpy.random.uniform(-lim, lim) + (1-int(real))*1.j * numpy.random.uniform(-lim, lim)
    sol[u_k] = numpy.random.uniform(-lim, lim) + (1-int(real))*1.j * numpy.random.uniform(-lim, lim)
    sol[a_i] = numpy.random.uniform(-lim, lim) + (1-int(real))*1.j * numpy.random.uniform(-lim, lim)
    sol[a_j] = numpy.random.uniform(-lim, lim) + (1-int(real))*1.j * numpy.random.uniform(-lim, lim)
    sol[a_k] = numpy.random.uniform(-lim, lim) + (1-int(real))*1.j * numpy.random.uniform(-lim, lim)
    return sol

def multiaffine(func):
    """calculate all second derivatives to test if something is multiaffine"""
    return [diff(func,v,2).simplify_full() for v in varlist]

def intfields(action, sol):
    return [ round(imaginary(-N(diff(action, var).subs(sol)/pi/2))) for var in paramvarlist]

class Quad:
    def __init__(self, Lshort, Llong, name="Q"):
        self.Lshort = Lshort
        self.Llong = Llong
        self.name = name
        
        self.phi = lambda u,v,a,b: diff(Llong(u,v,a,b),u)
        self.psi = lambda u,v,a: diff(Lshort(u,v,a),u)

        self.L3 = lambda u,u_i,u_j,u_ij,a,b: Lshort(u,u_i,a) - Lshort(u,u_j,b) - Llong(u_i,u_j,a,b)
        self.L4 = lambda u,u_i,u_j,u_ij,a,b: Lshort(u,u_i,a) - Lshort(u,u_j,b) - Llong(u,u_ij,a,b)
    
        self.S4 = cyclic( -self.L4(u,u_i,u_j,u_ij,a_i,a_j) + self.L4(u_k,u_ik,u_jk,u_ijk,a_i,a_j) )
        self.S3 = cyclic( -self.L3(u,u_i,u_j,u_ij,a_i,a_j) + self.L3(u_k,u_ik,u_jk,u_ijk,a_i,a_j) )
        
    def quads(self,a=0,b=8):
        """returns tetrahedron and quad equations"""
        return [diff(self.S4,v) for v in varlist[a:b]]
    # these are not in multiaffine form
    # attribute affinequads needs to be set manually
    
    def affinequads(self):
        return allquads(self.Q)
    
    def solve_cube(self,s_init):
        sol = s_init
        sol[u_ij] = u_ij.subs(solve(self.affinequads()[4], u_ij)).subs(sol)
        sol[u_jk] = u_jk.subs(solve(self.affinequads()[5], u_jk)).subs(sol)
        sol[u_ik] = u_ik.subs(solve(self.affinequads()[6], u_ik)).subs(sol)
        sol[u_ijk] = u_ijk.subs(solve(self.affinequads()[1], u_ijk)).subs(sol)
        return sol
        
    def biquadratic(self,v,w):
        bq = expand( ( self.Q*diff(self.Q,v,w)-diff(self.Q,v)*diff(self.Q,w) )  )
        if self.name in klist:
            bq = bq*klist[self.name]
        return factor(bq)
    
    def corners(self):
        """returns multiaffine corner eqns for the triangle Lagrangian"""
        return [diff(self.affinequads()[4],u)*self.affinequads()[6] - diff(self.affinequads()[6],u)*self.affinequads()[4],
                diff(self.affinequads()[5],u)*self.affinequads()[4] - diff(self.affinequads()[4],u)*self.affinequads()[5],
                diff(self.affinequads()[6],u)*self.affinequads()[5] - diff(self.affinequads()[5],u)*self.affinequads()[6],
                diff(self.affinequads()[1],u_ijk)*self.affinequads()[2] - diff(self.affinequads()[2],u_ijk)*self.affinequads()[1],
                diff(self.affinequads()[2],u_ijk)*self.affinequads()[3] - diff(self.affinequads()[3],u_ijk)*self.affinequads()[2],
                diff(self.affinequads()[3],u_ijk)*self.affinequads()[1] - diff(self.affinequads()[1],u_ijk)*self.affinequads()[3]]
    
    def S3EL(self,a=0,b=8):
        """returns corner equations of 3-point lagrangian"""
        return [diff(self.S3,v) for v in varlist[a:b]]
    
    def checkCAC(self, verbose=True):
        """checks CAC and prints formulas for double and triple shifts"""
        step1 = solve(self.affinequads()[4], u_ij) + solve(self.affinequads()[5], u_jk) + solve(self.affinequads()[6], u_ik)
        ans1 = u_ijk.subs(solve(self.affinequads()[1].subs(step1).simplify_trig(), u_ijk))
        ans2 = u_ijk.subs(solve(self.affinequads()[2].subs(step1).simplify_trig(), u_ijk))
        ans3 = u_ijk.subs(solve(self.affinequads()[3].subs(step1).simplify_trig(), u_ijk))
        if verbose:
            show(step1)
            show(ans1)
        return bool((expand(ans1-ans2).simplify_trig() == 0) and (expand(ans2-ans3).simplify_trig() == 0))
    
    def R(self):
        return [expand(1/2*( diff(self.corners()[0],u_i) + diff(self.corners()[4],u_jk) )),
                expand(1/2*( diff(self.corners()[1],u_j) + diff(self.corners()[5],u_ik) )),
                expand(1/2*( diff(self.corners()[2],u_k) + diff(self.corners()[3],u_ij) ))]
    def f(self):
        return [diff(self.affinequads()[4],u,u_i),diff(self.affinequads()[5],u,u_j),diff(self.affinequads()[6],u,u_k)]
    def Omega1(self):
        return (self.R()[0]/self.f()[0]).collect(a_i).collect(a_j).collect(a_k)
    def P1(self):
        return expand(self.corners()[0] - u_i*self.R()[0])
    def P23(self):
        return expand(self.corners()[4] - u_jk*self.R()[0])
    def g(self):
        out = -1 #TODO: adapt to individual eqns
        if self.name=="Q1_0":
            out = -a_i*(a_j+a_k-a_i)
        return out
    def Omega2(self):
        return ((self.P1() - self.P23())/self.g()).simplify_full().expand().collect(a_i).collect(a_j).collect(a_k)

### Defining the ABS Lagrangians in various transformations

In [23]:
lin = Quad(
    lambda u,v,a: u*v,
    lambda u,v,a,b: 1/2*(a-b)/(a+b)*(u-v)^2,
    "lin"
    )
H1 = Quad(
    lambda u,v,a: 2*u*v,
    lambda u,v,a,b: (a-b)*log((u-v)^2),
    "H1"
)
H1asym = Quad(
    lambda u,v,a: u*v,
    lambda u,v,a,b: (a-b)*log(u-v),
    "H1asym"
)
H2 = Quad(
    lambda u,v,a: (u+v+a)*log(u+v+a),
    lambda u,v,a,b: (u-v+a-b)*log(u-v+a-b) + (-u+v+a-b)*log(-u+v+a-b) + (u+v)*pi*I,
    "H2"
)
H2asym = Quad(
    lambda u,v,a: (u+v+a)*log(u+v+a),
    lambda u,v,a,b: (u-v+a-b)*log(u-v+a-b) + (-u+v+a-b)*log(u-v-a+b),
    "H2asym"
)
H3_0 = Quad(
    lambda u,v,a: -1/2*log(u*v)^2,
    lambda u,v,a,b: -dilog(a/b*u/v) - dilog(a/b*v/u) - 1/2*(log(-u/v))^2,
    "H3_0"
)
H3_0t = Quad(
    lambda u,v,a: -u*v,
    lambda u,v,a,b: -dilog(exp(a-b)*exp(u-v)) - dilog(exp(a-b)*exp(v-u)) - 1/2*(u-v)^2 + (u+v)*pi*I,
    "H3_0t"
)
H3_0asym = Quad(
    lambda u,v,a: -u*v,
    lambda u,v,a,b: -dilog(exp(a-b)*exp(u-v)) + dilog(exp(b-a)*exp(u-v)) - (a-b)*(u-v),
    "H3_0asym"
)
H3_1 = Quad(
    lambda u,v,a: dilog(-u*v/a) - log(a)*log(u*v),
    lambda u,v,a,b: -dilog(a/b*u/v) - dilog(a/b*v/u) - 1/2*(log(-u/v))^2, 
    "H3_1"
)
H3_1t = Quad(
    lambda u,v,a: dilog(exp(u+v-a)) - (a+pi*I)*(u+v),
    lambda u,v,a,b: -dilog(exp(a-b+u-v)) - dilog(exp(a-b+v-u)) - 1/2*(u-v)^2 + (u+v)*pi*I,
    "H3_1t"
)
H3_1asym = Quad(
    lambda u,v,a: dilog(exp(u+v-a)) - a*(u+v),
    lambda u,v,a,b: -dilog(exp(a-b+u-v)) + dilog(exp(b-a-v+u)) - (a-b)*(u-v),
    "H3_1asym"
)
Q1_0 = Quad(
    lambda u,v,a: a*log((u-v)^2),
    lambda u,v,a,b: (a-b)*log((u-v)^2),
    "Q1_0"
)
Q1_0asym = Quad(
    lambda u,v,a: a*log(u-v),
    lambda u,v,a,b: (a-b)*log(u-v),
    "Q1_0asym"
)
Q1_1 = Quad(
    lambda u,v,a: (u-v+a)*log(u-v+a) + (-u+v+a)*log(-u+v+a) + (u+v)*pi*I,
    lambda u,v,a,b: (u-v+a-b)*log(u-v+a-b) + (-u+v+a-b)*log(-u+v+a-b) + (u+v)*pi*I,
    "Q1_1"
)
Q1_1asym = Quad(
    lambda u,v,a: (u-v+a)*log(u-v+a) + (-u+v+a)*log(u-v-a),
    lambda u,v,a,b: (u-v+a-b)*log(u-v+a-b) + (-u+v+a-b)*log(u-v-a+b),
    "Q1_1asym"
)
Q2 = Quad(
    lambda u,v,a: (sqrt(u)+sqrt(v)+a)*log(sqrt(u)+sqrt(v)+a) + (sqrt(u)-sqrt(v)+a)*log(sqrt(u)-sqrt(v)+a)
                - (sqrt(u)+sqrt(v)-a)*log(sqrt(u)+sqrt(v)-a) + (-sqrt(u)+sqrt(v)+a)*log(-sqrt(u)+sqrt(v)+a) + (sqrt(u)+sqrt(v))*pi*I ,
    lambda u,v,a,b: (sqrt(u)+sqrt(v)+a-b)*log(sqrt(u)+sqrt(v)+a-b) + (sqrt(u)-sqrt(v)+a-b)*log(sqrt(u)-sqrt(v)+a-b) 
                - (sqrt(u)+sqrt(v)-a+b)*log(sqrt(u)+sqrt(v)-a+b) + (-sqrt(u)+sqrt(v)+a-b)*log(-sqrt(u)+sqrt(v)+a-b) + (sqrt(u)+sqrt(v))*pi*I ,
    "Q2"
)
Q2t = Quad(
    lambda u,v,a: (u+v+a)*log(u+v+a) + (u-v+a)*log(u-v+a) - (u+v-a)*log(u+v-a) + (-u+v+a)*log(-u+v+a) + (u+v)*pi*I ,
    lambda u,v,a,b: (u+v+a-b)*log(u+v+a-b) + (u-v+a-b)*log(u-v+a-b) - (u+v-a+b)*log(u+v-a+b) + (-u+v+a-b)*log(-u+v+a-b) + (u+v)*pi*I ,
    "Q2t"
)
Q2asym = Quad(
    lambda u,v,a: (u+v+a)*log(u+v+a) + (u-v+a)*log(u-v+a) + (-u-v+a)*log(u+v-a) + (-u+v+a)*log(u-v-a),
    lambda u,v,a,b: (u+v+a-b)*log(u+v+a-b) + (u-v+a-b)*log(u-v+a-b) + (-u-v+a-b)*log(u+v-a+b) + (-u+v+a-b)*log(u-v-a+b),
    "Q2asym"
)
Q3_0 = Quad( # Rational version
    lambda u,v,a: -dilog(a*u/v) - dilog(a*v/u) - 1/2*log(-u/v)^2, # -dilog(a*u/v) + dilog(u/a/v) - log(a)*log(u/v),
    lambda u,v,a,b: -dilog(a*u/b/v) - dilog(a*v/b/u) - 1/2*log(-u/v)^2, # -dilog(a*u/b/v) + dilog(b*u/a/v) - log(a/b)*log(u/v),
    "Q3_0"
)
Q3_0t = Quad(
    lambda u,v,a: -dilog(exp(a+u-v)) - dilog(exp(a+v-u)) - 1/2*(u-v)^2 + (u+v)*pi*I,
    lambda u,v,a,b: -dilog(exp((a-b)+u-v)) - dilog(exp((a-b)+v-u)) - 1/2*(u-v)^2 + (u+v)*pi*I,
    "Q3_0t"
)
Q3_0asym = Quad(
    lambda u,v,a: -dilog(exp(a+u-v)) + dilog(exp(-a+u-v)) - a*(u-v),
    lambda u,v,a,b: -dilog(exp((a-b)+u-v)) + dilog(exp(-(a-b)+u-v)) - (a-b)*(u-v),
    "Q3_0asym"
)
Q3_1 = Quad(
    lambda u,v,a: 0, # No explicit L known for the multiaffine form of Q3_1
    lambda u,v,a,b: 0,
    "Q3_1"
)
Q3_1t = Quad(
    lambda u,v,a: dilog(exp(+u+v-a)) + dilog(exp(+u-v-a)) 
                + dilog(exp(-u-v-a)) + dilog(exp(-u+v-a)) + u^2 + v^2 - a*(u-v),
    lambda u,v,a,b: dilog(exp(+u+v-a+b)) + dilog(exp(+u-v-a+b)) 
                    + dilog(exp(-u-v-a+b)) + dilog(exp(-u+v-a+b)) + u^2 + v^2 - (a-b)*(u-v),
    "Q3_1t"
)
Q3_1asym = Quad(
    lambda u,v,a: dilog(exp(-u+v-a)) + dilog(exp(-u-v-a)) 
                - dilog(exp(-u+v+a)) - dilog(exp(-u-v+a)) + a*(u-v),
    lambda u,v,a,b: dilog(exp(-u+v-a+b)) + dilog(exp(-u-v-a+b)) 
                    - dilog(exp(-u+v+a-b)) - dilog(exp(-u-v+a-b)) + (a-b)*(u-v) ,
    "Q3_1asym"
)
Q4 = Quad(
    lambda u,v,a: 0,
    lambda u,v,a,b: 0,
    "Q4"
)
A1_0 = Quad(
    lambda u,v,a: 2*a*log(u+v),
    lambda u,v,a,b: (a-b)*log((u-v)^2),
    "A1_0"
)
A1_0asym = Quad(
    lambda u,v,a: a*log(u+v),
    lambda u,v,a,b: (a-b)*log(u-v),
    "A1_0asym"
)
A1_1 = Quad(
    lambda u,v,a: (u+v+a)*log(u+v+a) + (-u-v+a)*log(u+v-a),
    lambda u,v,a,b: (-u+v+a-b)*log(-u+v+a-b) + (u-v+a-b)*log(u-v+a-b) + (u+v)*pi*I,
    "A1_1"
)
A1_1asym = Quad(
    lambda u,v,a: (u+v+a)*log(u+v+a) - (u+v-a)*log(u+v-a),
    lambda u,v,a,b: (-u+v+a-b)*log(-u+v+a-b) + (u-v+a-b)*log(-u+v-a+b),
    "A1_1asym"
)
A2 = Quad(
    lambda u,v,a: -dilog(a*u*v) + dilog(u*v/a) - log(a)*log(u*v),
    lambda u,v,a,b: -dilog(u/v*a/b) - dilog(v/u*a/b) - 1/2*log(-u/v)^2,
    "A2"
)
A2t = Quad(
    lambda u,v,a: -dilog(exp(a+u+v)) + dilog(exp(-a+u+v)) - a*(u+v),
    lambda u,v,a,b: -dilog(exp((a-b)+u-v)) - dilog(exp((a-b)+v-u)) - 1/2*(u-v)^2 + (u+v)*pi*I,
    "A2t"
)
A2asym = Quad(
    lambda u,v,a: -dilog(exp(a+u+v)) + dilog(exp(-a+u+v)) - a*(u+v),
    lambda u,v,a,b: -dilog(exp((a-b)+u-v)) + dilog(exp(-(a-b)+u-v)) - (a-b)*(u-v),
    "A2asym"
)

# ABS list including both multi-affine and transformed equations
ABSlist = [H1,H2,H3_0,H3_0t,H3_1,H3_1t,Q1_0,Q1_1,Q2,Q2t,Q3_0,Q3_0t,Q3_1,Q3_1t,A1_0,A1_1,A2,A2t]

# ABS list in multi-affine form
affABSlist = [H1,H2,H3_0,H3_1,Q1_0,Q1_1,Q2,Q3_0,Q3_1,A1_0,A1_1,A2]

# ABS list in transformed symmetric form
tABSlist = [H1,H2,H3_0t,H3_1t,Q1_0,Q1_1,Q2t,Q3_0t,Q3_1t,A1_0,A1_1,A2t]

# ABS list with asymetric Lagrangians for transformed equations
asymABSlist = [H1asym,H2asym,H3_0asym,H3_1asym,Q1_0asym,Q1_1asym,Q2asym,Q3_0asym,Q3_1asym,A1_0asym,A1_1asym,A2asym]

### Manual definition of multi-affine form of quad equations
Includes check that they are consistend with the multi-time Euler-Lagrange equations

In [24]:
# Here we set Quad.Q using case by case computations.
def polynomial(fraction, rhs=0):
    return fraction.numerator() - rhs*fraction.denominator()

lin.Q = -(a_i - a_j)*(u - u_ij)/(a_i + a_j) + u_i - u_j

eqn = H1
print(eqn.name)
quad = (u_i-u_j)*(u-u_ij) - a_i + a_j # manually define Q
print(expand( quad - polynomial(eqn.quads()[4]) )) # check that Q is multi-affine version of quad
print(expand( shiftk(2*quad) + polynomial(eqn.quads()[3]) ))
eqn.Q = quad

eqn = H1asym
print(eqn.name)
quad = (u_i-u_j)*(u-u_ij) - a_i + a_j # manually define Q
print(expand( quad - polynomial(eqn.quads()[4]) )) # check that Q is multi-affine version of quad
print(expand( shiftk(quad) + polynomial(eqn.quads()[3]) ))
eqn.Q = quad

eqn = H2
print(eqn.name)
quad = (u_i-u_j)*(u-u_ij) + (a_j-a_i)*(u+u_i+u_j+u_ij) - a_i^2 + a_j^2 # manually define Q
print( expand(quad - polynomial( exp(eqn.quads()[4]).simplify_full() ,1) )) # check that Q is multi-affine version of quad
print(expand( shiftk(quad) - polynomial( exp(eqn.quads()[3]).simplify_full() ,1) ))
eqn.Q = quad

eqn = H2asym
print(eqn.name)
quad = (u_i-u_j)*(u-u_ij) + (a_j-a_i)*(u+u_i+u_j+u_ij) - a_i^2 + a_j^2 # manually define Q
print( expand(quad - polynomial( exp(eqn.quads()[4]).simplify_full() ,1) )) # check that Q is multi-affine version of quad
print(expand( shiftk(quad) - polynomial( exp(eqn.quads()[3]).simplify_full() ,1) ))
eqn.Q = quad

eqn = H3_0
print(eqn.name)
quad = a_i*(u*u_i + u_j*u_ij) - a_j*(u*u_j + u_i*u_ij) # manually define Q
print(expand(quad + expand(polynomial( exp(expand(eqn.quads()[4]*u_ij)).simplify_full() ,1)) )) # check that Q is multi-affine version of quad
print(expand( shiftk(quad) - expand(polynomial( exp(expand(eqn.quads()[3]*u_k)).simplify_full() ,1)) ))
eqn.Q = quad

eqn = H3_0t
print(eqn.name)
quad = exp(a_i)*(e^u*e^u_i + e^u_j*e^u_ij) - exp(a_j)*(e^u*e^u_j + e^u_i*e^u_ij) # manually define Q
print(expand(quad + expand(polynomial( exp(expand(eqn.quads()[4])).simplify_full() ,1)) )) # check that Q is multi-affine version of quad
print(expand( shiftk(quad) + expand(polynomial( exp(expand(eqn.quads()[3])).simplify_full() ,1)) ))
eqn.Q = quad

eqn = H3_0asym
print(eqn.name)
quad = exp(a_i)*(e^u*e^u_i + e^u_j*e^u_ij) - exp(a_j)*(e^u*e^u_j + e^u_i*e^u_ij) # manually define Q
print(expand(quad + expand(polynomial( exp(expand(eqn.quads()[4])).simplify_full() ,1)) )) # check that Q is multi-affine version of quad
print(expand( shiftk(quad) + expand(polynomial( exp(expand(eqn.quads()[3])).simplify_full() ,1)) ))
eqn.Q = quad

eqn = H3_1
print(eqn.name)
quad = a_i*(u*u_i + u_j*u_ij) - a_j*(u*u_j + u_i*u_ij) + (a_i^2 - a_j^2) # manually define Q
print(expand(quad + expand(polynomial( exp(expand(eqn.quads()[4]*u_ij)).simplify_full() ,1)/u_ij) )) # check that Q is multi-affine version of quad
print(expand( shiftk(quad) - expand(polynomial( exp(expand(eqn.quads()[3]*u_k)).simplify_full() ,1)/u_k) )) # check that Q is multi-affine version of quad
eqn.Q = quad

eqn = H3_1t
print(eqn.name)
quad = -exp(a_i)*(e^u*e^u_i + e^u_j*e^u_ij) + exp(a_j)*(e^u*e^u_j + e^u_i*e^u_ij) + (exp(2*a_i) - exp(2*a_j)) # manually define Q
print(expand(quad - expand(polynomial( exp(expand(eqn.quads()[4])).simplify_full() ,1)*exp(-u_ij)) )) # check that Q is multi-affine version of quad
print(expand( shiftk(quad) + expand(polynomial( exp(expand(eqn.quads()[3])).simplify_full() ,1)*exp(-u_k)) )) # check that Q is multi-affine version of quad
eqn.Q = quad

eqn = H3_1asym
print(eqn.name)
quad = -exp(a_i)*(e^u*e^u_i + e^u_j*e^u_ij) + exp(a_j)*(e^u*e^u_j + e^u_i*e^u_ij) + (exp(2*a_i) - exp(2*a_j)) # manually define Q
print(expand(quad - expand(polynomial( exp(expand(eqn.quads()[4])).simplify_full() ,1)*exp(-u_ij)) )) # check that Q is multi-affine version of quad
print(expand( shiftk(quad) + expand(polynomial( exp(expand(eqn.quads()[3])).simplify_full() ,1)*exp(-u_k)) )) # check that Q is multi-affine version of quad
eqn.Q = quad

eqn = Q1_0
print(eqn.name)
quad = a_i*(u-u_j)*(u_i-u_ij) - a_j*(u-u_i)*(u_j-u_ij)
print(expand(2*quad + polynomial(eqn.quads()[4]) )) # check that Q is multi-affine version of quad
print(expand( shiftk(2*quad) + polynomial(eqn.quads()[3]) )) 
eqn.Q = quad

eqn = Q1_0asym
print(eqn.name)
quad = a_i*(u-u_j)*(u_i-u_ij) - a_j*(u-u_i)*(u_j-u_ij)
print(expand(quad + polynomial(eqn.quads()[4]) )) # check that Q is multi-affine version of quad
print(expand( shiftk(quad) + polynomial(eqn.quads()[3]) )) 
eqn.Q = quad

eqn = Q1_1
print(eqn.name)
quad = a_i*(u-u_j)*(u_i-u_ij) - a_j*(u-u_i)*(u_j-u_ij) + a_i*a_j*(a_i-a_j)
print(expand(quad + 1/2*polynomial(exp(eqn.quads()[4]).simplify_full() ,1) )) # check that Q is multi-affine version of quad
print(expand( shiftk(quad) + 1/2*polynomial(exp(eqn.quads()[3]).simplify_full() ,1) ))
eqn.Q = quad

eqn = Q1_1asym
print(eqn.name)
quad = a_i*(u-u_j)*(u_i-u_ij) - a_j*(u-u_i)*(u_j-u_ij) + a_i*a_j*(a_i-a_j)
print(expand(quad + 1/2*polynomial(exp(eqn.quads()[4]).simplify_full() ,1) )) # check that Q is multi-affine version of quad
print(expand( shiftk(quad) + 1/2*polynomial(exp(eqn.quads()[3]).simplify_full() ,1) ))
eqn.Q = quad

eqn = Q2
print(eqn.name)
quad = a_i*(u-u_j)*(u_i-u_ij) - a_j*(u-u_i)*(u_j-u_ij)
quad += a_i*a_j*(a_i-a_j)*(u + u_i + u_j + u_ij - a_i^2 + a_i*a_j - a_j^2) # manually define Q
print(expand(quad + polynomial( exp(2*eqn.quads()[4]*sqrt(u_ij)).simplify_full() ,1)/sqrt(u_ij)/4 )) # check that Q is multi-affine version of quad
print(expand( shiftk(quad) - polynomial( exp(2*eqn.quads()[3]*sqrt(u_k)).simplify_full() ,1)/sqrt(u_k)/4 )) 
eqn.Q = quad

eqn = Q2t
print(eqn.name)
quad = a_i*(u^2-u_j^2)*(u_i^2-u_ij^2) - a_j*(u^2-u_i^2)*(u_j^2-u_ij^2)
quad += a_i*a_j*(a_i-a_j)*(u^2 + u_i^2 + u_j^2 + u_ij^2 - a_i^2 + a_i*a_j - a_j^2) # manually define Q
print(expand(quad + polynomial( exp(eqn.quads()[4]).simplify_full() ,1)/4/u_ij )) # check that Q is multi-affine version of quad
print(expand( shiftk(quad) - polynomial( exp(eqn.quads()[3]).simplify_full() ,1)/4/u_k )) 
eqn.Q = quad

eqn = Q2asym
print(eqn.name)
quad = a_i*(u^2-u_j^2)*(u_i^2-u_ij^2) - a_j*(u^2-u_i^2)*(u_j^2-u_ij^2)
quad += a_i*a_j*(a_i-a_j)*(u^2 + u_i^2 + u_j^2 + u_ij^2 - a_i^2 + a_i*a_j - a_j^2) # manually define Q
print(expand(quad + polynomial( exp(eqn.quads()[4]).simplify_full() ,1)/4/u_ij )) # check that Q is multi-affine version of quad
print(expand( shiftk(quad) - polynomial( exp(eqn.quads()[3]).simplify_full() ,1)/4/u_k )) 
eqn.Q = quad

eqn = Q3_0
print(eqn.name)
quad = (a_j^2-a_i^2)*(u*u_ij + u_i*u_j) + a_j*(a_i^2-1)*(u*u_i + u_j*u_ij) - a_i*(a_j^2-1)*(u*u_j + u_i*u_ij) # manually define Q
print(expand(quad + polynomial(exp(eqn.quads()[4]*u_ij).simplify_full() ,1)/u_ij )) # check that Q is multi-affine version of quad
print(expand( shiftk(quad) + polynomial(exp(eqn.quads()[3]*u_k).simplify_full() ,1)/u_k ))
eqn.Q = quad

eqn = Q3_0t
print(eqn.name)
quad = -sinh(a_i-a_j)*(e^u*e^u_ij + e^u_i*e^u_j) + sinh(a_i)*(e^u*e^u_i + e^u_j*e^u_ij) - sinh(a_j)*(e^u*e^u_j + e^u_i*e^u_ij) # manually define Q
quad = sageobj(quad._maxima_().exponentialize())
t1 = expand( polynomial( exp(expand(eqn.quads()[4]) ).simplify_full() , 1)*exp(-u_ij)*exp(-a_i-a_j) )
print( expand(2*quad + t1) )
quadk = shiftk(quad)
t1 = expand( polynomial( exp(expand(eqn.quads()[3]) ).simplify_full() , 1)*exp(-u_k)*exp(-a_i-a_j) )
print( expand(2*quadk - t1) )
eqn.Q = quad

eqn = Q3_0asym
print(eqn.name)
quad = -sinh(a_i-a_j)*(e^u*e^u_ij + e^u_i*e^u_j) + sinh(a_i)*(e^u*e^u_i + e^u_j*e^u_ij) - sinh(a_j)*(e^u*e^u_j + e^u_i*e^u_ij) # manually define Q
quad = sageobj(quad._maxima_().exponentialize())
t1 = expand( polynomial( exp(expand(eqn.quads()[4]) ).simplify_full() , 1)*exp(-u_ij)*exp(-a_i-a_j) )
print( expand(2*quad + t1) )
quadk = shiftk(quad)
t1 = expand( polynomial( exp(expand(eqn.quads()[3]) ).simplify_full() , 1)*exp(-u_k)*exp(-a_i-a_j) )
print( expand(2*quadk - t1) )
eqn.Q = quad

eqn = Q3_1
print(eqn.name)
quad = -(a_i^2-a_j^2)*(u*u_ij + u_i*u_j) + a_j*(a_i^2-1)*(u*u_i + u_j*u_ij) - a_i*(a_j^2-1)*(u*u_j + u_i*u_ij) - (a_j^2-a_i^2)*(a_i^2-1)*(a_j^2-1)/a_i/a_j # manually define Q
eqn.Q = quad

eqn = Q3_1t
print(eqn.name)
quad = -sinh(a_i-a_j)*(cosh(u)*cosh(u_ij) + cosh(u_i)*cosh(u_j)) 
quad += sinh(a_i)*(cosh(u)*cosh(u_i) + cosh(u_j)*cosh(u_ij)) 
quad += -sinh(a_j)*(cosh(u)*cosh(u_j) + cosh(u_i)*cosh(u_ij))
quad += -sinh(a_i-a_j)*sinh(a_i)*sinh(a_j) # manually define Q
t1 = expand( 16*quad*sinh(u_ij) )
t1 = expand( sageobj(t1._maxima_().exponentialize()) )
t2 = expand(polynomial( exp(eqn.quads()[4]).simplify_full() , 1)/exp(u+u_i+u_j+3*u_ij+2*a_i+2*a_j))
print(expand( t1 - t2 ))
t1 = expand( 16*shiftk(quad)*sinh(u_k) )
t1 = expand( sageobj(t1._maxima_().exponentialize()) )
t2 = expand(polynomial( exp(eqn.quads()[3]).simplify_full() , 1)/exp(3*u_k+u_ik+u_jk+u_ijk+2*a_i+2*a_j))
print(expand( t1 + t2 ))
eqn.Q = quad

eqn = Q3_1asym
print(eqn.name)
quad = -sinh(a_i-a_j)*(cosh(u)*cosh(u_ij) + cosh(u_i)*cosh(u_j)) 
quad += sinh(a_i)*(cosh(u)*cosh(u_i) + cosh(u_j)*cosh(u_ij)) 
quad += -sinh(a_j)*(cosh(u)*cosh(u_j) + cosh(u_i)*cosh(u_ij))
quad += -sinh(a_i-a_j)*sinh(a_i)*sinh(a_j) # manually define Q
t1 = expand( 16*quad*sinh(u_ij) )
t1 = expand( sageobj(t1._maxima_().exponentialize()) )
t2 = expand(polynomial( exp(eqn.quads()[4]).simplify_full() , 1)/exp(u+u_i+u_j+3*u_ij+2*a_i+2*a_j))
print(expand( t1 - t2 ))
t1 = expand( 16*shiftk(quad)*sinh(u_k) )
t1 = expand( sageobj(t1._maxima_().exponentialize()) )
t2 = expand(polynomial( exp(eqn.quads()[3]).simplify_full() , 1)/exp(3*u_k+u_ik+u_jk+u_ijk+2*a_i+2*a_j))
print(expand( t1 + t2 ))
eqn.Q = quad

eqn = Q4
print(eqn.name)
var("a,b,c,A,B,C")
quad =  A*( (u-b)*(u_j-b) - (a-b)*(c-b) ) * ( (u_i-b)*(u_ij-b) - (a-b)*(c-b) )
quad += B*( (u-a)*(u_i-a) - (b-a)*(c-a) ) * ( (u_j-a)*(u_ij-a) - (b-a)*(c-a) ) - A*B*C*(a-b)
eqn.Q = quad

eqn = A1_0
print(eqn.name)
quad = a_i*(u*u_i + u_j*u_ij) - a_j*(u*u_j + u_i*u_ij) + (a_i-a_j)*(u_i*u_j + u*u_ij) # manually define Q
print(expand(2*quad + polynomial( eqn.quads()[4]) )) # check that Q is multi-affine version of quad
print(expand( shiftk(2*quad) - polynomial( eqn.quads()[3]) ))
eqn.Q = quad

eqn = A1_0asym
print(eqn.name)
quad = a_i*(u*u_i + u_j*u_ij) - a_j*(u*u_j + u_i*u_ij) + (a_i-a_j)*(u_i*u_j + u*u_ij) # manually define Q
print(expand(quad + polynomial( eqn.quads()[4]) )) # check that Q is multi-affine version of quad
print(expand( shiftk(quad) - polynomial( eqn.quads()[3]) ))
eqn.Q = quad

eqn = A1_1
print(eqn.name)
quad = a_i*(u*u_i + u_j*u_ij) - a_j*(u*u_j + u_i*u_ij) + (a_i-a_j)*(u_i*u_j + u*u_ij) - a_i*a_j*(a_i-a_j) # manually define Q
print(expand(quad - polynomial(exp(eqn.quads()[4]).simplify_full(),1)/2 )) # check that Q is multi-affine version of quad
print(expand( shiftk(quad) - polynomial(exp(eqn.quads()[3]).simplify_full(),1)/2 ))
eqn.Q = quad

eqn = A1_1asym
print(eqn.name)
quad = a_i*(u*u_i + u_j*u_ij) - a_j*(u*u_j + u_i*u_ij) + (a_i-a_j)*(u_i*u_j + u*u_ij) - a_i*a_j*(a_i-a_j) # manually define Q
print(expand(quad - polynomial(exp(eqn.quads()[4]).simplify_full(),1)/2 )) # check that Q is multi-affine version of quad
print(expand( shiftk(quad) - polynomial(exp(eqn.quads()[3]).simplify_full(),1)/2 ))
eqn.Q = quad

eqn = A2
print(eqn.name)
quad = -(a_i^2-a_j^2)*(1+u*u_i*u_j*u_ij) - a_i*(a_j^2-1)*(u*u_i + u_j*u_ij) + a_j*(a_i^2-1)*(u*u_j + u_i*u_ij) # manually define Q
t1 = expand( polynomial( exp(expand(eqn.quads()[4]*u_ij) ).simplify_full() , 1)/u_ij )
print( expand(quad - t1) )
t1 = expand( polynomial( exp(expand(eqn.quads()[3]*u_k) ).simplify_full() , 1)/u_k )
print( expand(shiftk(quad) + t1) )
eqn.Q = quad

eqn = A2t
print(eqn.name)
quad = -sinh(a_i-a_j)*(1+e^u*e^u_i*e^u_j*e^u_ij) - sinh(a_j)*(e^u*e^u_i + e^u_j*e^u_ij) + sinh(a_i)*(e^u*e^u_j + e^u_i*e^u_ij) # manually define Q
quad = 2*sageobj(quad._maxima_().exponentialize())
t1 = expand( polynomial( exp(expand(eqn.quads()[4]) ).simplify_full() , 1)*exp(-u_ij)*exp(-a_i-a_j) )
print( expand(quad - t1) )
quadk = shiftk(quad)
t1 = expand( polynomial( exp(expand(eqn.quads()[3]) ).simplify_full() , 1)*exp(-u_k)*exp(-a_i-a_j) )
print( expand(quadk + t1) )
eqn.Q = quad

eqn = A2asym
print(eqn.name)
quad = -sinh(a_i-a_j)*(1+e^u*e^u_i*e^u_j*e^u_ij) - sinh(a_j)*(e^u*e^u_i + e^u_j*e^u_ij) + sinh(a_i)*(e^u*e^u_j + e^u_i*e^u_ij) # manually define Q
quad = 2*sageobj(quad._maxima_().exponentialize())
t1 = expand( polynomial( exp(expand(eqn.quads()[4]) ).simplify_full() , 1)*exp(-u_ij)*exp(-a_i-a_j) )
print( expand(quad - t1) )
quadk = shiftk(quad)
t1 = expand( polynomial( exp(expand(eqn.quads()[3]) ).simplify_full() , 1)*exp(-u_k)*exp(-a_i-a_j) )
print( expand(quadk + t1) )
eqn.Q = quad

H1
0
0
H1asym
0
0
H2
0
0
H2asym
0
0
H3_0
0
0
H3_0t
0
0
H3_0asym
0
0
H3_1
0
0
H3_1t
0
0
H3_1asym
0
0
Q1_0
0
0
Q1_0asym
0
0
Q1_1
0
0
Q1_1asym
0
0
Q2
0
0
Q2t
0
0
Q2asym
0
0
Q3_0
0
0
Q3_0t
0
0
Q3_0asym
0
0
Q3_1
Q3_1t
0
0
Q3_1asym
0
0
Q4
A1_0
0
0
A1_0asym
0
0
A1_1
0
0
A1_1asym
0
0
A2
0
0
A2t
0
0
A2asym
0
0


### Print the biquadratics for each equation of the ABS list

In [25]:
klist = {"H1": -1/(a_i-a_j), "H2": -1/2/(a_i-a_j), "H3_0": 1/(a_i^2-a_j^2), "H3_1": 1/(a_i^2-a_j^2), 
         "Q1_0": 1/(a_i-a_j)/a_i/a_j, "Q1_1": -1/(a_i-a_j)/a_i/a_j, "Q2": -1/(a_i-a_j)/a_i/a_j,
         "Q3_0": 1/(a_i^2-a_j^2)/(a_i^2-1)/(a_j^2-1), "Q3_1": 1/(a_i^2-a_j^2)/(a_i^2-1)/(a_j^2-1),
         "A1_0": 1/(a_i-a_j)/a_i/a_j, "A1_1": 1/(a_i-a_j)/a_i/a_j, "A2": 1/(a_i^2-a_j^2)/(a_i^2-1)/(a_j^2-1)}

for eq in affABSlist:
    print(eq.name)
    show(eq.affinequads()[4])
    show(eq.biquadratic(u_j,u_ij).simplify_rational())
    show(eq.biquadratic(u_j,u_i).simplify_rational())
    print()

H1


(u - u_ij)*(u_i - u_j) - a_i + a_j

1

-(u^2 - 2*u*u_ij + u_ij^2)/(a_i - a_j)


H2


...

-(a_i^2 - 2*a_i*a_j + a_j^2 - u^2 + 2*u*u_ij - u_ij^2)/(a_i - a_j)


A2


-(a_j^2 - 1)*(u*u_i + u_ij*u_j)*a_i + (a_i^2 - 1)*(u_i*u_ij + u*u_j)*a_j - (u*u_i*u_ij*u_j + 1)*(a_i^2 - a_j^2)

(a_i*u^2*u_i^2 - (a_i^2 + 1)*u*u_i + a_i)/(a_i^2 - 1)

(a_i*a_j*u^2 + a_i*a_j*u_ij^2 - (a_i^2 + a_j^2)*u*u_ij)/(a_i^2 - a_j^2)

### Print the leg functions and Lagrangians for each equation of the ABS list

In [26]:
for eq in asymABSlist:
    print(eq.name)
    show(eq.psi(u,u_i,a_i))
    show(eq.phi(u,u_ij,a_i,a_j))
    show(eq.Lshort(u,u_i,a_i))
    show(eq.Llong(u,u_ij,a_i,a_j))
    print()

H1asym


u_i

(a_i - a_j)/(u - u_ij)

u*u_i

(a_i - a_j)*log(u - u_ij)

...


A2asym


-a_i + log(-e^(a_i + u + u_i) + 1) - log(-e^(-a_i + u + u_i) + 1)

-a_i + a_j + log(-e^(a_i - a_j + u - u_ij) + 1) - log(-e^(-a_i + a_j + u - u_ij) + 1)

-a_i*(u + u_i) - dilog(e^(a_i + u + u_i)) + dilog(e^(-a_i + u + u_i))

-(a_i - a_j)*(u - u_ij) - dilog(e^(a_i - a_j + u - u_ij)) + dilog(e^(-a_i + a_j + u - u_ij))

In [27]:
for eq in tABSlist:
    print(eq.name)
    show(eq.psi(u,u_i,a_i))
    show(eq.phi(u,u_ij,a_i,a_j))
    show(eq.Lshort(u,u_i,a_i))
    show(eq.Llong(u,u_ij,a_i,a_j))
    print()

H1


2*u_i

2*(a_i - a_j)/(u - u_ij)

2*u*u_i

(a_i - a_j)*log((u - u_ij)^2)

...


A2t


-a_i + log(-e^(a_i + u + u_i) + 1) - log(-e^(-a_i + u + u_i) + 1)

I*pi - u + u_ij + log(-e^(a_i - a_j + u - u_ij) + 1) - log(-e^(a_i - a_j - u + u_ij) + 1)

-a_i*(u + u_i) - dilog(e^(a_i + u + u_i)) + dilog(e^(-a_i + u + u_i))

I*pi*(u + u_ij) - 1/2*(u - u_ij)^2 - dilog(e^(a_i - a_j + u - u_ij)) - dilog(e^(a_i - a_j - u + u_ij))

### Numerically test closure on random solutions

Only non-zero results are printed.

First for the fully-symmetric Lagrangians:

In [7]:
numpy.random.seed(1)

def absfloor(x):
    return sign(x)*floor(abs(x))

def find_exponents(expr, s=0, im=False):
    expr = str(expr)
    out = []
    while expr.find("e^") > 0:
        expr = expr[expr.find("e^"):]
        end = expr.find(")")
        expo = eval(expr[3:end])
        if s == 0:
            out += [expo]
        elif im:
            out += [absfloor(imaginary(N(expo.subs(s)))/2/pi)] 
        else:
            out += [expo.subs(s).n(digits=4)]                
        expr = expr[end:]
    if out == []:
        out += [0]
    return out

for eqn in [lin] + tABSlist:
    print(eqn.name)
    for i in range(20):
        if eqn == H3_0t:
            s = H3_0.solve_cube(randominit())
            for v in paramvarlist: s[v] = log(s[v])
        elif eqn == H3_1t:
            s = H3_1.solve_cube(randominit())
            for v in paramvarlist: s[v] = log(s[v])
            for a in [a_i,a_j,a_k]: s[a] = s[a]+pi*I
        elif eqn == Q2t:
            s = Q2.solve_cube(randominit())
            for v in varlist: s[v] = sqrt(s[v])
        elif eqn == Q3_0t:
            s = Q3_0.solve_cube(randominit())
            for v in paramvarlist: s[v] = log(s[v])
        elif eqn == A2t:
            s = A2.solve_cube(randominit())
            for v in paramvarlist: s[v] = log(s[v])
        else:
            s = eqn.solve_cube(randominit())
        thetas = intfields(eqn.S4, s) # [ round(imaginary(-N(diff(eqn.S4, var).subs(s)/pi/2))) for var in paramvarlist]
        extS = eqn.S4 + 2*pi*I*( sum([thetas[i]*paramvarlist[i] for i in range(len(paramvarlist))]))
        closure = N(extS.subs(s))
        if abs(closure) > 1e-10:
            print( closure.n(digits=4) )
    print()

lin

H1

H2

H3_0t
-39.48 + 1.776e-15*I
-39.48 + 7.105e-15*I
39.48 + 7.105e-15*I

H3_1t
39.48 - 5.204e-16*I
-39.48 - 4.219e-15*I
-39.48 - 3.553e-15*I
-39.48 + 9.326e-15*I
-39.48 - 3.553e-15*I
39.48 - 1.332e-15*I

Q1_0

Q1_1

Q2t

Q3_0t
-39.48 - 1.776e-15*I
39.48 + 6.661e-15*I
-39.48
-39.48

Q3_1t
-78.96 - 1.421e-14*I
157.9
-78.96 + 3.553e-14*I
-78.96 - 2.842e-14*I
78.96 - 2.132e-14*I
39.48 + 1.243e-14*I
-39.48 + 2.842e-14*I
-118.4 + 2.842e-14*I
-39.48 + 1.421e-14*I
-118.4 - 7.283e-14*I
-39.48 + 1.421e-14*I
-39.48 - 3.730e-14*I
-39.48
39.48 + 2.842e-14*I
-39.48 - 7.105e-15*I
39.48 - 1.776e-14*I

A1_0

A1_1

A2t
39.48 - 2.165e-15*I
39.48 - 7.105e-15*I
-39.48 + 1.776e-15*I
39.48 + 7.105e-15*I
-39.48 - 7.105e-15*I
39.48 + 3.553e-15*I



Then for the simpler non-symmetric Lagrangians:

In [8]:
numpy.random.seed(2)

for eqn in asymABSlist:
    print(eqn.name)
    for i in range(20):
        if eqn == H3_0asym:
            s = H3_0.solve_cube(randominit())
            for v in paramvarlist: s[v] = log(s[v])
        elif eqn == H3_1asym:
            s = H3_1.solve_cube(randominit())
            for v in paramvarlist: s[v] = log(s[v])
            for a in [a_i,a_j,a_k]: s[a] = s[a]+pi*I
        elif eqn == Q2asym:
            s = Q2.solve_cube(randominit())
            for v in varlist: s[v] = sqrt(s[v])
        elif eqn == Q3_0asym:
            s = Q3_0.solve_cube(randominit())
            for v in paramvarlist: s[v] = log(s[v])
        elif eqn == A2asym:
            s = A2.solve_cube(randominit())
            for v in paramvarlist: s[v] = log(s[v])
        else:
            s = eqn.solve_cube(randominit())
        thetas = intfields(eqn.S4, s)
        extS = eqn.S4 + 2*pi*I*( sum([thetas[i]*paramvarlist[i] for i in range(len(paramvarlist))]))
        closure = N(extS.subs(s))
        if abs(closure) > 1e-10:
            print( closure.n(digits=4) )
    print()

H1asym

H2asym

H3_0asym
-39.48 - 8.882e-16*I
-39.48 - 9.992e-16*I
-39.48 + 1.776e-15*I
-39.48 - 3.553e-15*I
39.48 - 1.776e-15*I
39.48 - 1.776e-15*I
39.48
39.48 + 8.882e-16*I

H3_1asym
-39.48 + 3.664e-15*I
39.48 + 3.331e-15*I
-39.48 - 6.217e-15*I
39.48 + 3.109e-15*I
-39.48 + 6.661e-15*I
78.96 + 1.110e-15*I
-39.48 + 3.997e-15*I
-39.48 + 1.776e-15*I
39.48

Q1_0asym

Q1_1asym

Q2asym

Q3_0asym
39.48 + 2.665e-15*I
-39.48 + 1.776e-15*I
39.48 - 4.441e-15*I
-39.48 + 3.553e-15*I
-39.48 + 1.776e-15*I
39.48 - 1.332e-15*I

Q3_1asym
39.48 - 4.263e-14*I
-39.48 + 4.441e-15*I
39.48 + 1.421e-14*I
78.96 - 1.066e-14*I
-39.48 - 1.066e-14*I
78.96 - 1.421e-14*I
-39.48 - 7.550e-15*I

A1_0asym

A1_1asym

A2asym
-39.48 - 1.332e-15*I
78.96 + 3.553e-15*I
39.48 + 7.105e-15*I
-39.48 + 7.105e-15*I
39.48 + 2.665e-15*I
-39.48 + 1.776e-15*I
-39.48 - 1.776e-15*I



For H1 triangle Lagrangian with random weak solution

In [9]:
eqn = H1asym

numpy.random.seed(1)

for i in range(5):
    init = randominit()
    init[u_ij] = init[u]
    del init[u]
    s = init

    s_octa = solve([eqn.corners()[0].subs(init),eqn.corners()[1].subs(init)],u_ik,u_jk)
    for eq in s_octa[0]:
        s[eq.lhs()] = N(eq.rhs())

    thetas = intfields(eqn.S3, s)
    extS = eqn.S3 + 2*pi*I*( sum([thetas[i]*paramvarlist[i] for i in range(len(paramvarlist))]))
    closure = N(extS.subs(s))
    
    print( closure.n(digits=4) )
    print(intfields(eqn.S3, s))
    print()

-3.553e-15 - 3.553e-15*I
[0, 1, -1, 0, 0, 0, 0, 0, 0, 0, 0]

7.994e-15 - 1.776e-15*I
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

5.329e-15 + 7.105e-15*I
[1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0]

-1.776e-15
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

0.0000
[-1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]



### Further checks: CAC

In [10]:
for q in affABSlist: 
    print(q.name)
    print(q.checkCAC(verbose=False))
    print()

H1
True

H2
True

H3_0
True

H3_1
True

Q1_0
True

Q1_1
True

Q2
True

Q3_0
True

Q3_1
True

A1_0
True

A1_1
True

A2
True



### Lemma 9: initial data of the form 1+O(t)

In [11]:
var("w,w_i,w_j")
var("c_i,c_j")
var("t")
expansion = {u:1+w*t, u_i:1+w_i*t, u_j:1+w_j*t, a_i:1+c_i*t^2, a_j:1+c_j*t^2}
for eq in affABSlist:
    print(eq.name)
    doubleshift = u_ij.subs(solve(eq.Q,u_ij)).subs(expansion).simplify_rational().series(t,2)
    show( doubleshift.coefficient(t,0).simplify_full() )
    show( doubleshift.coefficient(t,1).simplify_full() )

H1


1

(w*w_i - w*w_j - c_i + c_j)/(w_i - w_j)

H2


1

...

1

(w*w_i - w*w_j + 3*c_i - 3*c_j)/(w_i - w_j)

A2


1

-(c_i*w*w_i - (c_j*w - (c_i - c_j)*w_i)*w_j)/((c_i - c_j)*w - c_j*w_i + c_i*w_j)

# Explicit examples

### H2, part 1: random solutions: 2 π i terms in three-leg form

In [12]:
eqn = H2

for i in range(10):
    s = eqn.solve_cube(randominit())
    print( ( (eqn.psi(u,u_i,a_i) - eqn.psi(u,u_j,a_j) - eqn.phi(u,u_ij,a_i,a_j)).subs(s)).n(digits=4) )
    print( ( (eqn.psi(u_ij,u_j,a_i) - eqn.psi(u_ij,u_i,a_j) - eqn.phi(u_ij,u,a_i,a_j)).subs(s) ).n(digits=4) )
    print()

0.0000
-6.283*I

-4.441e-16 - 6.283*I
0.0000

-6.283*I
2.810e-16

-8.882e-16 - 6.283*I
1.554e-15 - 6.283*I

-4.441e-16 - 6.283*I
6.661e-16

0.0000
-6.283*I

8.882e-16 - 6.283*I
2.220e-16

2.220e-16
-2.220e-16 - 6.283*I

-2.220e-16
-2.220e-16 - 6.283*I

4.441e-16 + 6.283*I
-5.551e-17 - 12.57*I



### H1, part 5: explicit example with S = ± 2 π i without adding Ξ

In [13]:
eqn = H1asym

print(eqn.Llong(u,u_ij,a_i,a_j))
print()

init = {a_i: 1, a_j: 2, a_k: 3, u: 0, u_i: 1, u_j: I, u_k: 1+I}
s = eqn.solve_cube(init)
thetas = intfields(eqn.S4, s)

print(s)
print()

print(thetas)
print()

print( (diff(eqn.S4,a_k).subs(s)).n(digits=4) )
print( (eqn.S4.subs(s)).n(digits=4) )
print( (eqn.S3.subs(s)).n(digits=4) )

(a_i - a_j)*log(u - u_ij)

{a_i: 1, a_j: 2, a_k: 3, u: 0, u_i: 1, u_j: I, u_k: I + 1, u_ij: 1/2*I + 1/2, u_jk: -1, u_ik: 2*I, u_ijk: 3/5*I + 6/5}

[0, -1, 1, 0, 0, 0, 0, 0, 0, 0, 0]

-6.283*I
0.00001526 - 6.283*I
-0.00001144 + 6.283*I


### H3_0, part 3: explicit example with S = -4 pi^2

In [14]:
eqn = H3_0asym

init = {u: exp(1 - I), u_i: exp(1 + I), u_j: exp(I),
        u_k: exp(2 + I), a_i: exp(2 - 2*I), a_j: exp(1 + 2*I), a_k: exp(1)}

s = H3_0.solve_cube(init)
for v in paramvarlist: s[v] = N(log(s[v]))
print(s)
print()

thetas = intfields(eqn.S4, s)
print(thetas)
extS = eqn.S4 + 2*pi*I*( sum([thetas[i]*paramvarlist[i] for i in range(len(paramvarlist))]))
closure = (extS.subs(s)).n()
print(closure)

{u: 1.00000000000000 - 1.00000000000000*I, u_i: 1.00000000000000 + 1.00000000000000*I, u_j: 1.00000000000000*I, u_k: 2.00000000000000 + 1.00000000000000*I, a_i: 2.00000000000000 - 2.00000000000000*I, a_j: 1.00000000000000 + 2.00000000000000*I, a_k: 1.00000000000000, u_ij: 1.49110840612501 + 1.80621078461947*I, u_jk: 1.00000000000000 + 3.05123312020969*I, u_ik: 0.459012522305983 + 1.68677242027984*I, u_ijk: 1.05436866494958 - 2.73523350791485*I}

[0, 0, 0, -1, 0, 0, 1, 1, 0, 0, -1]
-39.4784176043574 + 1.77635683940025e-15*I


### H2, part 2: neighbouring cubes require different Θ

In [15]:
eqn = H2

init = {a_i:1-I, a_j: -I, a_k: 1, u: 0, u_i: 1, u_j: 1+I, u_k: I}
s = eqn.solve_cube(init)
thetas = intfields(eqn.S4, s)

si = {a_i: s[a_i], a_j: s[a_j], a_k: s[a_k]}
si[u] = s[u_i]
si[u_j] = s[u_ij]
si[u_k] = s[u_ik]
si[u_i] = 1
si = eqn.solve_cube(si)

thetasi = intfields(eqn.S4, si)

print(s)
print(si)
print()

print(paramvarlist)
print(thetas)
print(thetasi)

{a_i: -I + 1, a_j: -I, a_k: 1, u: 0, u_i: 1, u_j: I + 1, u_k: I, u_ij: -I - 2, u_jk: I - 3, u_ik: 3/5*I - 6/5, u_ijk: 3/5*I + 6/5}
{a_i: -I + 1, a_j: -I, a_k: 1, u: 1, u_j: -I - 2, u_k: 3/5*I - 6/5, u_i: 1, u_ij: 14/17*I + 12/17, u_jk: 3/5*I + 6/5, u_ik: 45/37*I + 11/37, u_ijk: 1/2*I - 17/6}

[a_i, a_j, a_k, u, u_i, u_j, u_k, u_ij, u_jk, u_ik, u_ijk]
[0, -1, 1, -1, 1, 0, 1, 0, -1, -1, 1]
[-1, 1, 0, -1, 1, 0, 1, -1, 0, -1, 1]
